# MTH 651: Advanced Numerical Analysis

## Homework Assignment 3

### (Student Name 👈 Write Your Name Here!)

### Guidelines

* Each student must complete their own assignment individually.
  * Discussing with other students is allowed (encouraged!), but you must write your own answers and not copy off of others.
* Submit the homework as a Jupyter notebook with **properly formatted LaTeX**


### Bilinear Quadrilateral Element

Consider the **unit square** $[0,1]^2$.

We will define the **bilinear element** on the unit square.

The space of shape functions is $\mathcal{Q}_1 = \operatorname{span}\{ 1, x, y, xy \}$.

Let $v_1, v_2, v_3, v_4$ denote the vertices of the square.

Let $N_i(u) : u \mapsto u(v_i)$ denote the corresponding **nodal variables**.

#### Problem 1. (2 points)

Create the Vandermonde matrix corresponding to the basis $\{ 1, x, y, xy \}$ and nodal variables $\{ N_i \}$.

In [ ]:
# Define the Vandermonde matrix V here

#### Problem 2. (2 points)

Use the Vandermonde matrix to find the **nodal basis** for $\mathcal{Q}_1$ (i.e. the basis **dual** to the nodal variables).

Write out (i.e. as closed form expressions) the formulas for the basis functions.

In [ ]:
# Use V to find the nodal basis functions

### Biquadratic Quadrilateral Element

Now consider the space $\mathcal{Q}_2 = \operatorname{span}\{ 1, x, x^2, y, xy, x^2 y, y^2, x y^2, x^2 y^2 \}$.

Define the **nodal points** $(0, 1/2, 1) \times (0, 1/2, 1)$ (here $\times$ represents the Cartesian product).

#### Problem 3. (2 points)

Create the Vandermonde matrix for the biquadratic quadrilateral element.

In [ ]:
# Define the Vandermonde matrix V here

#### Problem 4. (2 points)

Use the Vandermonde matrix to find the **nodal basis** for $\mathcal{Q}_2$ (i.e. the basis **dual** to the nodal variables).

Write out (i.e. as closed form expressions) the formulas for the basis functions.

In [ ]:
# Use V to find the nodal basis functions

#### Problem 5. (2 points)

In general, the space $\mathcal{Q}_k$ is the space of polynomials of degree at most $k$ **in each variable**.

* What is $\dim( \mathcal{Q}_k ([0,1]^2))$?
* What is $\dim( \mathcal{Q}_k ([0,1]^d))$?

#### Problem 6. (Bonus)

What are good **nodal variables** for the space $\mathcal{Q}_k$ on the unit square $[0,1]^2$?

#### Problem 7. (4 points)

Let the domain $\Omega$ be given as the union of subdomains $K_i$.

Suppose $u$ is piecewise $C^\infty$, i.e. $u|_{K_i} \in C^\infty(K_i)$ for all $i$.

Prove that $u \in H^1(\Omega)$ iff $u$ is continuous.

#### Hint:

* Let $\boldsymbol{n}_{K_i}$ denote the outward facing normal vector on $K_i$.
* Let $v \in L^2(\Omega)$.
* Let $e$ be an **interface**, i.e. $e = K_i \cap K_j \neq \varnothing$ for some $i$ and $j$.
* Let $\Gamma$ be the union of all such interfaces (called the **skeleton** of the mesh).
* Define the **jump** of $v$ by $[v] = v_{K_i} \boldsymbol{n}_{K_i} + v_{K_j} \boldsymbol{n}_{K_j}$.
    * (Note that $[v] = (v_{K_i} - v_{K_j}) \boldsymbol{n}_{K_i}$)

Then,

$$
    \sum_{i} \int_{\partial K_i} v \phi \boldsymbol{n} \, ds = \int_\Gamma [v] \phi \, ds
$$

If $\int_\Gamma [v] \phi \, ds = 0$ for all smooth test functions, what can be inferred about $[v]$ (and hence about $v$)?

#### Problem 8: MFEM Coding. (4 points)

We are going to try to measure the **rate of convergence** of **global interpolation** in a finite element space.

**Problem 8(a).** Read carefully and understand the following MFEM code snippets (if asked to do something similar in a future assignment, you should understand how to write code like this on your own).

In [ ]:
# Install MFEM in Colab
%pip install mfem
%pip install glvis

In [1]:
# Import MFEM
import mfem.ser as mfem

def compute_interpolation_error(n, order_k, f):
    """
    Compute and return the interpolation error (measured in L2 norm)

    Parameters:
        n (int): size of Cartesian grid in each dimension
        order_k (int): polynomial degree
        f (coefficient): the function to be approximated

    Returns:
        L2 norm of the interpolation error
    """
    # Create a triangular mesh of a n x n Cartesian grid.
    # The n^2 cells of the Cartesian grid will each be split into two triangles,
    # resulting in 2*n^2 triangles.
    mesh = mfem.Mesh(n, n, "TRIANGLE")
    # Define the type of finite element we will use. We will use continuous 
    # finite elements of degree at most `order_k`.
    fe_collection = mfem.H1_FECollection(order_k, mesh.Dimension())
    # "Glue" together the finite elements to create a space on the entire mesh.
    # This represents the discrete space V_h.
    fe_space = mfem.FiniteElementSpace(mesh, fe_collection)
    # Define a "grid function" on the space. A grid function represents an 
    # element u_h of the finite element space V_h.
    u = mfem.GridFunction(fe_space)
    # Project coefficient performs the *global interpolation* of the given 
    # function (which has to be represented as a "Coefficient object"). 
    # This sets the degrees of freedom based on evaluating the *nodal variables*
    # of the input function f.
    u.ProjectCoefficient(f)
    # Return the L2 norm || u_h - f ||.
    return u.ComputeL2Error(f)

MFEM has the concept of a "coefficient". A coefficient defines an function that can be evaluated at different points in the domain. (Essentially, "coefficient" is just the MFEM terminology for a user-specified function)

In [2]:
# Define a function f that we will try to approximate by interpolation
from math import cos, exp, sin
class f(mfem.PyCoefficient):
    def EvalValue(self, xvec):
        x = xvec[0]
        y = xvec[1]
        return exp(0.1*sin(5.1*x - 6.2*y) + 0.3*cos(4.3*x +3.4*y))

In [3]:
# Example of using the above functions:

compute_interpolation_error(10, 2, f())

0.00034178312291455767

**Problem 8(b).**

Start with a mesh with `n = 4`, and refine by a factor of two 4 times.

* Set $k = 1$ and compute the $L^2$ interpolation error and empirical order of convergence.
* Repeat the same task for $k = 2$ and $k = 3$.
* Based on the above, conjecture what the order of convergence should be for arbitrary $k$.